In [15]:
date = "2024-01-03"  # to get csv

In [16]:
import importlib
from pathlib import Path

from modules import module
import numpy as np
import pandas as pd

try:
    importlib.reload(module)  # reload module
except NameError:
    pass

# Load DF

In [17]:
path = Path(f"input/{date}.csv")
df = pd.read_csv(path)

target_cols = [
    "External ID",
    "Display Name",
    "Date of Birth",
    "Email",
    "Phone",
    "Contract Type",
    "From",
    "To",
    "Fully Paid Date",
    "Membership/Display Name",
    "Membership Status",
    "Partner/Branch/Display Name",
    "Partner/Date of Birth",
    "Partner/Age",
    "Partner/City",
    "Partner/Gender",
    "Partner/Household income/month",
    "Partner/Industry/Display Name",
    "Partner/Job Position",
    "Partner/Occupation",
    "Partner/Street",
    "Partner/Street2",
    "Followers (Partners)/Hobby",
    "Followers (Partners)/Interest",
    "Partner/Hobby",
    "Partner/Interest",
]
current_cols = list(df.columns)
map_cols = dict(zip(current_cols, target_cols))
df.columns = df.columns.map(map_cols)

print(df.shape)
df.sample(3)

(29338, 26)


,External ID,Display Name,Date of Birth,Email,Phone,Contract Type,From,To,Fully Paid Date,Membership/Display Name,...,Partner/Household income/month,Partner/Industry/Display Name,Partner/Job Position,Partner/Occupation,Partner/Street,Partner/Street2,Followers (Partners)/Hobby,Followers (Partners)/Interest,Partner/Hobby,Partner/Interest
8242,__export__.membership_membership_line_196460_f...,FERDY MARCHIS GUMARANG PASARIBU,2006-03-03,ferdimarchis@gmail.com,081281002374,Private,2022-02-13,2022-11-12,2022-02-10,[DLX.9] 9 Months Membership Deluxe - 2022 - CB,...,Rp 10.000.000 - Rp 20.000.000,NaN,'-,'-,Bumi Mutiara Blok JH 3/4,"Perumahan Bumi Mutiara Bojong Kulur,",NaN,NaN,watching movies,"Educational, youth development"
12741,__export__.membership_membership_line_184849_c...,YULIA FIRSTIANNISA,NaN,NaN,NaN,NaN,2019-07-01,2021-07-01,2019-09-29,[DLX.24] 24 Months Membership DLX - DG,...,Rp 10.000.000 - Rp 20.000.000,NaN,Owner,Makeup Artist,JL. Bunga Matahari VII Blok G.18 Kecamatan Pon...,NaN,NaN,NaN,"Culinary, Travelling, Makeup","Makeup, Western Food"
17721,__export__.membership_membership_line_168946_d...,ARIFA QONITA NAURA ABINDRA,NaN,NaN,NaN,NaN,2019-06-03,2019-12-03,NaN,NaN,...,NaN,NaN,NaN,NaN,"JL. CONDET BATU AMPAR V NO. 11, 13520",NaN,NaN,NaN,NaN,NaN


# Clean DF

In [18]:
df_clean = (
    df
    # drop na
    .dropna(subset="From")
    .dropna(subset="To")
    # drop date too old
    .loc[lambda df_: pd.to_datetime(df_["From"]).dt.year >= 2020]
    # drop membership code na
    .loc[lambda df_: ~(df_["Membership/Display Name"].isna())]
    # filter cancelled member and free member and non member
    .loc[
        lambda df_: ~(
            df_["Membership Status"].isin(
                ["Cancelled Member", "Free Member", "Non Member"]
            )
        )
    ]
    # filter staff
    .loc[lambda df_: df_["Contract Type"] != "Employee"]
    # rename column
    .rename(
        columns=lambda c: (
            c.lower()
            .replace("/display name", "")
            .replace("/month", "")
            .replace("position", "")
            .strip()
            .replace(" ", "_")
            .replace("/", "_")
        )
    )
    .rename(columns=module.to_rename)
    # drop unused cols
    .drop(columns=module.to_drop)
    .assign(
        # clean city
        city=lambda df_: (df_["city"].str.title().str.strip()),
        # clean dob
        dob=lambda df_: np.where(
            df_["dob"].isna(), pd.to_datetime(df_["dob2"]), pd.to_datetime(df_["dob"])
        ),
        # clean start_date, end_date, fp_date
        start_date=lambda df_: pd.to_datetime(df_["start_date"]),
        end_date=lambda df_: pd.to_datetime(df_["end_date"]),
        fp_date=lambda df_: pd.to_datetime(df_["fp_date"]),
        fp_month=lambda df_: df_["fp_date"].dt.strftime("%m / %b %Y"),
        fp_year=lambda df_: df_["fp_date"].dt.strftime("%Y"),
        # clean income
        income=lambda df_: df_["income"].astype(module.income_cat),
        # clean job
        job=lambda df_: np.where(
            df_["job1"].isna(),
            df_["job2"].str.lower().str.replace("[^\w\s]", "", regex=True).str.strip(),
            df_["job1"].str.lower().str.replace("[^\w\s]", "", regex=True).str.strip(),
        ),
        # create age
        age=lambda df_: (
            (df_["start_date"] - df_["dob"])
            .div(pd.Timedelta("365 days"))
            .apply(np.floor)
        ),
        # create id
        id=lambda df_: df_["name"] + " " + df_["dob"].astype(str),
        # create membership code
        membership_code=lambda df_: module.get_membership_code(df_["product"]),
        # create membership duration
        membership_duration=lambda df_: module.get_membership_duration(df_["product"]),
    )
    # merge with membership mapping to obtain membership
    .merge(
        right=pd.read_excel(Path("input/membership_mapping.xlsx")),
        on="membership_code",
        how="left",
    )
    # make sure that all corporate is mapped
    .assign(
        is_cpt=lambda df_: np.where(
            df_["product"].str.lower().str.contains("cpt|corporate|corp", regex=True),
            True,
            df_["is_cpt"],
        )
    )
    # create student center and area 
    .assign(
        center=lambda df_: module.clean_center(df_),
        area=lambda df_: module.clean_area(df_),
    )
    # sort by id
    .sort_values(["id", "end_date"], ascending=[True, False])
    # create is next contract col
    .assign(is_next_contract=lambda df_: df_["id"].duplicated(keep="last"))
    # drop unused columns
    .drop(columns=["dob2", "job1", "job2", "name"])
    # drop membership code na
    .loc[lambda df_: ~(df_["membership_code"].isna())]
    # sort column
    .sort_index(axis=1)
    # # ! create is_active
    .assign(
        active_jan_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 jan 2023"
        ),
        active_feb_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 feb 2023"
        ),
        active_mar_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 mar 2023"
        ),
        active_apr_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 apr 2023"
        ),
        active_may_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 may 2023"
        ),
        active_jun_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 jun 2023"
        ),
        active_jul_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 jul 2023"
        ),
        active_aug_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 aug 2023"
        ),
        active_sep_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 sep 2023"
        ),
        active_oct_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 oct 2023"
        ),
        active_nov_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 nov 2023"
        ),
        active_dec_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 dec 2023"
        ),
    )
)
df_clean.groupby(["area", "center"]).size()

area           center       
BDG            DG                953
Corporate      Corporate         568
JKT 1          PP               1479
               SDC               664
JKT 2          BSD               239
               CP                 25
               GC               1381
               LW                712
               TBS                 6
JKT 3          CBB               271
               KK               1510
               SMB               134
Online Center  Online Center    3251
Other          HO                117
               ID                 67
               NST              1432
SBY            PKW               394
dtype: int64

# Tests and Assertions

In [19]:
# assert that all centers are mapped
center_in_df = df["Partner/Branch/Display Name"].unique()

unmapped = []
for c in center_in_df:
    if isinstance(c, str) and c.upper() not in module.center_map.keys():
        unmapped.append(c)

if len(unmapped) > 0:
    print(unmapped)
    raise Exception("There are unmapped centers. Map this inside modules/center_map.")

In [20]:
# assert that all membership codes has ben accounted
codes = df_clean["membership_code"].unique()
maps = pd.read_excel(Path("input/membership_mapping.xlsx"))["membership_code"].unique()

unmapped = []
for code in codes:
    if code not in maps:
        if code == np.NaN:
            continue
        unmapped.append(code)

if len(unmapped) > 0:
    for i in unmapped:
        print(i)
    raise Exception("Some membership are not mapped.")

# Save DF

In [21]:
import os
file= df_clean
path= f'output/member_data_{date}.parquet'

if not os.path.exists(path):
    file.to_parquet(path)
    print('File saved.')
else:
    print('File already exist.')

File saved.


/home/anj/miniconda3/envs/data-science/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


# Experiment

In [22]:
# df.loc[df['Membership/Display Name'].str.lower().str.contains('ilt', na= False)].sort_values('From')

In [23]:
# df['Membership/Display Name'].unique()

In [24]:
# df_clean['is_cpt'].value_counts()

In [25]:
# df["Partner/Branch/Display Name"].unique()